The following code draws from the Ray docs at:
https://docs.ray.io/en/releases-1.11.0/ray-core/using-ray-with-pytorch.html

In [1]:
from filelock import FileLock
import argparse
from torchvision import datasets, transforms
import torch
import torch.optim as optim
import ray
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import numpy as np

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [4]:
def dataset_creator(use_cuda, data_dir):
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}
    with FileLock("./data.lock"):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(
                data_dir,
                train=True,
                download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),
            batch_size=128,
            shuffle=True,
            **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            data_dir,
            train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])),
        batch_size=128,
        shuffle=True,
        **kwargs)

    return train_loader, test_loader

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(
                output, target, reduction="sum").item()
            pred = output.argmax(
                dim=1,
                keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    return {
        "loss": test_loss,
        "accuracy": 100 * correct / len(test_loader.dataset)
    }


In [6]:
%run /Users/lucastucker/REU-2023/laplacian_eigenmaps_functions.ipynb
# REPLACE with your file path

In [8]:
"""
This method converts data from the torch loader into numpy, performs the 
laplacian eigenmaps dimension reduction specified in laplacian_eigenmaps_functions
and then returns the reduced loader
"""
def LE_on_loader(loader, t = 5, max_rad = 10, sigma = 0.2, proj_dim = 30):
    modified_samples = []        
    for data, target in loader:
        X = data.view(data.size(0), -1).numpy().T # numpy representation 784 x 128
        n, m = X.shape

        # This is baseline LE
        # """
        N = t_nearest_matr(m, t, X)
        W = weight_matr(m, N, X, sigma)
        P = get_le_reduced(proj_dim, X, W)
        # """

        # This is the ALE  
        """
        N = t_nearest_matr(m, max_rad, X)
        avgs = avg_distance_matr(m, t, X, N)
        W = averaged_laplacian_weight(m, t, X, N, avgs)
        P = get_le_reduced(proj_dim, X, W)
        """      

        # This is the novel VRLE
        """
        N = t_nearest_matr(m, max_rad, X)
        avgs = avg_distance_matr(m, max_rad, X, N)
        W = variable_nbrs_weight(m, max_rad, X, N, avgs)
        P = get_le_reduced(proj_dim, X, W)
        """
        print(f"the shape of P is {P.shape}")
        P = P.reshape((data.size(0), proj_dim))
        print(f"data.size(0) is {data.size(0)}")

        modified_samples.append((torch.from_numpy(P).float(), target))

    modified_data = [data for data, _ in modified_samples]
    modified_targets = [target for _, target in modified_samples]

    # Create a custom dataset to include both data and target
    modified_dataset = torch.utils.data.TensorDataset(torch.cat(modified_data, dim=0), torch.cat(modified_targets, dim=0))
    modified_loader = torch.utils.data.DataLoader(modified_dataset, batch_size=128, shuffle=True)

    return modified_loader

In [9]:
class Network(object):
    def __init__(self, lr=0.01, momentum=0.4, data_dir="~/data"):
        use_cuda = torch.cuda.is_available()
        self.device = device = torch.device("cuda" if use_cuda else "cpu")
        self.train_loader, self.test_loader = dataset_creator(use_cuda, data_dir)
        
        self.train_loader = LE_on_loader(self.train_loader)
        # for data, tgt in self.train_loader:
        #    print(f"size of data is {data.size()}")
        self.test_loader = LE_on_loader(self.test_loader)

        self.model = Model().to(device)
        self.optimizer = optim.SGD(
        self.model.parameters(), lr=lr, momentum=momentum)

    def train(self):
        train(self.model, self.device, self.train_loader, self.optimizer)
        return test(self.model, self.device, self.test_loader)

    def get_weights(self):
        return self.model.state_dict()

    def set_weights(self, weights):
        self.model.load_state_dict(weights)

    def save(self):
        torch.save(self.model.state_dict(), "mnist_cnn.pt")



In [10]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--data-dir",
    type=str,
    default="~/data/",
    help="Set the path of the dataset."
)
args, unknown = parser.parse_known_args()

net = Network(data_dir=args.data_dir)
net.train()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/lucastucker/data/MNIST/raw/train-images-idx3-ubyte.gz to /Users/lucastucker/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/lucastucker/data/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/lucastucker/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/lucastucker/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/lucastucker/data/MNIST/raw

the shape of P is (30, 128)
data.size(0) is 128


/var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_2326/4231425207.py:33: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:250.)
  modified_samples.append((torch.from_numpy(P).float(), target))


the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0) is 128
the shape of P is (30, 128)
data.size(0)

ValueError: cannot reshape array of size 256 into shape (16,30)

In [42]:
RemoteNetwork = ray.remote(Network) # convert Network class to Ray Actor class
NetworkActor = RemoteNetwork.remote() # Instantiate
NetworkActor2 = RemoteNetwork.remote()
NetworkActor3 = RemoteNetwork.remote()
NetworkActor4 = RemoteNetwork.remote()

ray.get([NetworkActor.train.remote(), NetworkActor2.train.remote(),
         NetworkActor3.train.remote(), NetworkActor4.train.remote()]) # Fetch Ray object reference (future)

RayActorError: The actor died because of an error raised in its creation task, [36mray::Network.__init__()[39m (pid=72238, ip=127.0.0.1, actor_id=066a9da27ea126be0edbc66301000000, repr=<__main__.Network object at 0x1102c5960>)
  File "/var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_69465/226944357.py", line 8, in __init__
ValueError: too many values to unpack (expected 2)

(Network pid=72238) /var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_69465/959501931.py:40: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:250.)
(Network pid=72238) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::Network.__init__() (pid=72238, ip=127.0.0.1, actor_id=066a9da27ea126be0edbc66301000000, repr=<__main__.Network object at 0x1102c5960>)
(Network pid=72238)   File "/var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_69465/226944357.py", line 8, in __init__
(Network pid=72238) ValueError: too many values to unpack (expected 2)


Results Baseline LE:
[{'loss': 0.1838333763062954, 'accuracy': 94.61},
 {'loss': 0.1724409951210022, 'accuracy': 95.21},
 {'loss': 0.1792655333995819, 'accuracy': 94.97},
 {'loss': 0.1930500351190567, 'accuracy': 94.33}]

Results ALE:
[{'loss': 0.1711803722858429, 'accuracy': 94.85},
 {'loss': 0.18033546368479728, 'accuracy': 94.99},
 {'loss': 0.18480656784176827, 'accuracy': 94.76},
 {'loss': 0.18867687324285506, 'accuracy': 94.52}]

Results VRLE:
[{'loss': 0.1841858439207077, 'accuracy': 94.6},
 {'loss': 0.17663083124160767, 'accuracy': 95.03},
 {'loss': 0.1818410368204117, 'accuracy': 94.47},
 {'loss': 0.16766286469101907, 'accuracy': 95.08}]

 Results Random Projection:



In [ ]:
!ray stop --force

Stopped only 14 out of 20 Ray processes within the grace period 16 seconds. Set `-v` to see more details. Remaining processes [psutil.Process(pid=47659, name='raylet', status='zombie', started='07:34:48'), psutil.Process(pid=47660, name='Python', status='zombie', started='07:34:48'), psutil.Process(pid=47652, name='Python', status='zombie', started='07:34:46'), psutil.Process(pid=47655, name='Python', status='zombie', started='07:34:47'), psutil.Process(pid=47656, name='Python', status='zombie', started='07:34:47'), psutil.Process(pid=47653, name='gcs_server', status='zombie', started='07:34:46')] will be forcefully terminated.
You can also use `--force` to forcefully terminate processes or set higher `--grace-period` to wait longer time for proper termination.
